# Loading the Data

First, we'll load some data to play with. Spark can load a variety of formats:

```
spark.read.text()
spark.read.json()
spark.read.format(<format goes here, e.g., csv>)
```

By default, it will attempt to automatically set up the schema if we are using dataframes / datasets.

**On a somewhat related note**, if you've just stored a lot of data in HDFS to use with Spark, then you might want to rebalance your cluster with:

```
hdfs balancer -threshold 1
```

This should ensure that blocks are spread evenly across the workers.

In [ ]:
df = spark.read.json('hdfs://orion11:35001/RC_2008-12.bz2')
#df = spark.read.json('hdfs://orion11:35001/RES-RC_2018-01.zst')

(let's wait for a bit while it loads our dataframe... Depending on how large the file is, it may take a while.)

When this finally finishes, we can look at the data, and more importantly, our schema:

In [ ]:
print(df.take(2))
print('There are {} records in this dataframe.'.format(df.count()))

In [ ]:
df.printSchema()

# Defining a Schema 

We can actually define a schema ahead of time, assuming it won't change. Let's say we have a schema:

```
schema = df.schema
```

Then we can use

```
spark.load.schema(schema).json('...')
```

To avoid all the overhead from generating the schema automatically. This can be a big speed boost.

However, be careful: in most datasets, the schema will change over time.

# Playing with the Data

In [ ]:
# Let's see if any scores are over 9000
df.filter(df.score > 9000).count()

In [ ]:
# How many posts were there in the politics subreddit?
df.filter(df.subreddit == "politics").count()

# SQL

In [ ]:
df.createOrReplaceTempView("df_view")

# How many different users posted in /r/politics this month?
spark.sql("SELECT distinct author FROM df_view WHERE subreddit = 'politics'").count()

In [ ]:
print("{} unique users posted {} times!".format(Out[11], Out[8]))

In [ ]:
# What is the highest score (+/- upvotes/downvotes) in /r/technology?
spark.sql("SELECT MAX(score) as max_score FROM df_view WHERE subreddit = 'technology'").collect()[0][0]
# NOTE: we get a 'row' result here, and index into it. It's safe to use .collect() here to transfer the data
# to the driver, but if there was more than one row we have to be very cautious!

# Working with processed data

In [ ]:
from pyspark.sql.functions import *


top_authors = df.groupBy("author").count().sort(col("count").desc())
top_authors.show()

In [ ]:
import pandas as pd
p = top_authors.limit(6).toPandas()
p = p.iloc[1: , :] # Drop the first row

Notice how we use 'limit' instead of take; limit returns a dataframe whereas 'take' is a terminal operation. Then we convert the limited dataframe to a Pandas dataframe.

**Note:** if you want to view some data, `.show()` tends to be nice (see above)

In [ ]:
p.plot.barh()
p